In [6]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

data_set= pd.read_csv("C:/Users/adity/OneDrive/Desktop/Sixth Semester/CE6018 Seismic Data Analytics/Program/sda_Python Program/data_set.csv")


In [1]:




def eemd(Y, Nstd, NE):
    xsize = len(Y)
    dd = np.arange(1, xsize + 1)
    Ystd = np.std(Y)
    Y = Y / Ystd

    TNM = int(np.fix(np.log2(xsize))) - 1
    TNM2 = TNM + 2

    allmode = np.zeros((xsize, TNM2))

    for iii in range(NE):
        X1 = Y + np.random.randn(xsize) * Nstd

        mode = np.zeros((xsize, TNM + 1))
        mode[:, 0] = Y

        xorigin = X1
        xend = xorigin

        nmode = 1
        while nmode <= TNM:
            xstart = xend.copy()
            iter_count = 1

            while iter_count <= 10:
                spmax, spmin = extrema(xstart)
                upper = interp1d(spmax[:, 0], spmax[:, 1], kind='linear', fill_value='extrapolate')(dd)
                lower = interp1d(spmin[:, 0], spmin[:, 1], kind='linear', fill_value='extrapolate')(dd)
                mean_ul = (upper + lower) / 2
                xstart = xstart - mean_ul
                iter_count += 1

            xend = xend - xstart
            nmode += 1

            mode[:, nmode] = xstart

        mode[:, nmode] = xend
        allmode += mode

    allmode = allmode / NE
    allmode = allmode * Ystd

    return allmode


def extrema(data):
    d = np.diff(data)
    maxima = (d[:-1] > 0) & (d[1:] <= 0)
    minima = (d[:-1] < 0) & (d[1:] >= 0)

    spmax = np.column_stack((np.where(maxima)[0] + 1, data[maxima]))
    spmin = np.column_stack((np.where(minima)[0] + 1, data[minima]))

    return spmax, spmin

# Example usage:
# Replace Y, Nstd, and NE with your data and parameters
# result = eemd(Y, Nstd, NE)
# print(result)


In [ ]:
from scipy.signal import hilbert
from scipy.fftpack import fftshift
from scipy.interpolate import interp2d
import matplotlib.pyplot as plt
from eemd import eemd


def EPSDParam(EWa, dt):
    bb = len(EWa)
    fs = 1 / dt

    StandevData = np.std(EWa)

    if StandevData > 50:
        NstdSet = [0.1, 0.2, 0.3, 0.5, 0.7]
    elif 10 <= StandevData <= 50:
        NstdSet = [0.01, 0.05, 0.09, 0.1, 0.2]
    elif 1 <= StandevData <= 10:
        NstdSet = [0.005, 0.009, 0.01, 0.05, 0.1]
    elif StandevData <= 1:
        NstdSet = [0.001, 0.003, 0.005, 0.009, 0.01]

    VarP = 0

    for n in range(len(NstdSet)):
        Nstd = NstdSet[n]
        allmode = eemd(EWa, Nstd, 3)
        varData = np.var(allmode[:, 0])
        lend, wid = allmode.shape
        PerVar = np.zeros((wid - 1,))

        for m in range(1, wid - 1):
            PerVar[m - 1] = (np.var(allmode[:, m]) / varData) * 100

        VarTotal = np.sum(PerVar)

        if 85 <= VarTotal <= 115:
            VarP = VarTotal
            break

    if VarP == 0:
        Nstd = 0.81
        allmode = eemd(EWa, Nstd, 3)
        varData = np.var(allmode[:, 0])
        lend, wid = allmode.shape
        PerVar = np.zeros((wid - 1,))

        for m in range(1, wid - 1):
            PerVar[m - 1] = (np.var(allmode[:, m]) / varData) * 100

        VarP = np.sum(PerVar)

    IMF = allmode[:, 1:]
    Time = bb * dt

    AF = np.zeros_like(IMF, dtype=np.complex128)
    Cj = np.abs(AF)
    Thetaj = np.unwrap(np.angle(AF))
    IF = np.diff(Thetaj, axis=0) / (2 * np.pi * dt)

    IF = np.concatenate((IF, np.zeros((1, wid - 2))))

    for k in range(1, wid - 1):
        for k1 in range(bb):
            if IF[k1, k] < 0:
                IF[k1, k] = 0

    Ws = np.linspace(0, fs / 2, wid - 1)
    Ts = np.linspace(0, Time, bb)
    GTW = np.zeros((bb, wid - 1))

    for x in range(bb):
        for imf in range(1, wid - 1):
            freqid = round(IF[x, imf], 1)
            freqidx = int(np.where(np.round(Ws, 1) == freqid)[0])

            if 1 <= freqidx <= len(Ws):
                tx = x
                GTW[tx, freqidx] += Cj[x, imf] ** 2 * 0.5

    Ws, Ts = np.meshgrid(Ws, Ts)
    GTW = np.flipud(np.rot90(GTW))

    Param = {
        'GTW': GTW,
        'Ws': Ws,
        'Ts': Ts
    }

    return Param

# Example usage:
# Replace EWa and dt with your data
# result = EPSDParam(EWa, dt)
# print(result)
# plt.pcolormesh(result['Ts'], result['Ws'], result['GTW'])
# plt.xlabel('Time')
# plt.ylabel('Frequency')
# plt.show()


In [ ]:

def extrema(in_data):
    flag = 1
    dsize = len(in_data)

    spmax = np.zeros((2, 1))
    spmax[0, 0] = 1
    spmax[1, 0] = in_data[0]
    jj = 2
    kk = 2

    while jj < dsize:
        if (in_data[jj - 1] <= in_data[jj]) and (in_data[jj] >= in_data[jj + 1]):
            spmax = np.append(spmax, [[jj], [in_data[jj]]], axis=1)
            kk += 1
        jj += 1

    spmax = np.append(spmax, [[dsize], [in_data[dsize - 1]]], axis=1)

    if kk >= 4:
        slope1 = (spmax[1, 1] - spmax[2, 1]) / (spmax[1, 0] - spmax[2, 0])
        tmp1 = slope1 * (spmax[0, 0] - spmax[1, 0]) + spmax[1, 1]

        if tmp1 > spmax[0, 1]:
            spmax[0, 1] = tmp1

        slope2 = (spmax[kk - 1, 1] - spmax[kk - 2, 1]) / (spmax[kk - 1, 0] - spmax[kk - 2, 0])
        tmp2 = slope2 * (spmax[kk, 0] - spmax[kk - 1, 0]) + spmax[kk - 1, 1]

        if tmp2 > spmax[kk, 1]:
            spmax[kk, 1] = tmp2
    else:
        flag = -1

    msize = in_data.shape
    dsize = max(msize)
    xsize = dsize // 3
    xsize2 = 2 * xsize

    spmin = np.zeros((2, 1))
    spmin[0, 0] = 1
    spmin[1, 0] = in_data[0]
    jj = 2
    kk = 2

    while jj < dsize:
        if (in_data[jj - 1] >= in_data[jj]) and (in_data[jj] <= in_data[jj + 1]):
            spmin = np.append(spmin, [[jj], [in_data[jj]]], axis=1)
            kk += 1
        jj += 1

    spmin = np.append(spmin, [[dsize], [in_data[dsize - 1]]], axis=1)

    if kk >= 4:
        slope1 = (spmin[1, 1] - spmin[2, 1]) / (spmin[1, 0] - spmin[2, 0])
        tmp1 = slope1 * (spmin[0, 0] - spmin[1, 0]) + spmin[1, 1]

        if tmp1 < spmin[0, 1]:
            spmin[0, 1] = tmp1

        slope2 = (spmin[kk - 1, 1] - spmin[kk - 2, 1]) / (spmin[kk - 1, 0] - spmin[kk - 2, 0])
        tmp2 = slope2 * (spmin[kk, 0] - spmin[kk - 1, 0]) + spmin[kk - 1, 1]

        if tmp2 < spmin[kk, 1]:
            spmin[kk, 1] = tmp2
    else:
        flag = -1

    return spmax, spmin, flag

# Example usage:
# Replace in_data with your actual data
# spmax, spmin, flag = extrema(in_data)
# print("spmax:", spmax)
# print("spmin:", spmin)
# print("flag:", flag)


In [ ]:
def FASp(dt, xgtt):
    # Nyquist frequency (highest frequency)
    Ny = (1 / dt) / 2
    # number of points in xgtt
    L = len(xgtt)
    # Next power of 2 from length of xgtt
    NFFT = 2**int(np.ceil(np.log2(L)))
    # frequency spacing
    df = 1 / (NFFT * dt)
    # Fourier amplitudes
    U = np.abs(np.fft.fft(xgtt, NFFT)) * dt
    # Single sided Fourier amplitude spectrum
    U = U[1:int(Ny/df) + 1]
    # frequency range
    f = np.linspace(df, Ny, int(Ny/df))

    return f, U

# Example usage:
# Replace dt and xgtt with your actual values
# dt = 0.005
# xgtt = np.random.randn(1000)  # Replace with your data
# f, U = FASp(dt, xgtt)
# print("f:", f)
# print("U:", U)


In [ ]:
def kohmachi(signal, freq_array, smooth_coeff):
    ii = np.arange(1, len(freq_array) + 1)
    
    ratio = np.repeat(freq_array, len(freq_array)).reshape(-1, len(freq_array)) / freq_array[ii - 1].reshape(-1, 1)
    rat = np.log10(ratio)
    
    w = (np.sin(rat * smooth_coeff) / (rat * smooth_coeff))**4
    w[np.isnan(w)] = 0
    
    s = np.sum(w[:, ii - 1], axis=1)
    avg = w.T / s
    y = avg[ii - 1] @ signal
    
    return y

# Example usage:
# Replace signal, freq_array, and smooth_coeff with your actual values
# signal = np.random.randn(1000, 3)  # Replace with your data
# freq_array = np.linspace(1, 10, 1000)
# smooth_coeff = 0.1
# result = kohmachi(signal, freq_array, smooth_coeff)
# print(result)


In [ ]:
def responsespectrum(accel, ee, dt):
    Tn = 10
    y = 0.5
    b = 0.25
    uo = 0
    vo = 0
    m = 1
    z = ee / 100
    
    na = len(accel)
    nl = 2 * na
    
    T = np.array([0.01, 0.015, 0.02, 0.03, 0.04, 0.05, 0.06, 0.075, 0.09, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75,
                  0.8, 0.9, 1, 1.2, 1.5, 2, 2.5, 3, 4, 5, 6, 7.5, 8, 9, 10])
    
    accel = np.concatenate((accel, np.zeros(nl - na)))
    p = -m * accel
    
    A = np.zeros(len(T))  # acceleration response spectrum - total acceleration
    V = np.zeros(len(T))  # velocity response spectrum - relative velocity
    D = np.zeros(len(T))  # displacement response spectrum - relative displacement
    
    for j in range(len(T)):
        fn = 1 / T[j]
        wn = 2 * np.pi * fn
        k = m * wn**2
        c = 2 * m * wn * z
        
        u = np.zeros(nl)
        v = np.zeros(nl)
        ac = np.zeros(nl)
        
        u[0] = uo
        v[0] = vo
        ac[0] = (p[0] - c * vo - k * uo) / m
        
        kf = k + y * c / (b * dt) + m / (b * dt**2)
        a = m / (b * dt) + y * c / b
        b2 = m / (2 * b) + dt * (y / (2 * b) - 1) * c
        
        for i in range(nl - 1):
            p1 = p[i]
            p2 = p[i + 1]
            dpf = (p2 - p1) + a * v[i] + b2 * ac[i]
            du = dpf / kf
            dv = y / (b * dt) * du - (y / b) * v[i] + dt * (1 - y / (2 * b)) * ac[i]
            da = du / (b * dt**2) - v[i] / (b * dt) - ac[i] / (2 * b)
            u[i + 1] = u[i] + du
            v[i + 1] = v[i] + dv
            ac[i + 1] = ac[i] + da
        
        asd = ac + accel
        A[j] = np.max(np.abs(asd))
        V[j] = np.max(np.abs(v))
        D[j] = np.max(np.abs(u))
    
    A = np.concatenate(([np.max(np.abs(accel))], A))
    V = np.concatenate(([0], V))
    D = np.concatenate(([0], D))
    
    PSV = (2 * np.pi / T) * D[1:]  # pseudo spectral velocity
    PSV = np.concatenate(([PSV[0]], PSV))
    PSA = ((2 * np.pi / T)**2) * D[1:]  # pseudo spectral acceleration
    PSA = np.concatenate(([PSA[0]], PSA))
    
    T = np.concatenate(([0], T))
    
    plt.figure()
    plt.plot(T, A)
    plt.xlabel('Time Period (seconds)')
    plt.ylabel('Spectral Acceleration')
    
    plt.figure()
    plt.plot(T, V)
    plt.xlabel('Time Period (seconds)')
    plt.ylabel('Spectral Velocity')
    
    plt.figure()
    plt.plot(T, D)
    plt.xlabel('Time Period (seconds)')
    plt.ylabel('Spectral Displacement')
    
    plt.figure()
    plt.plot(T, PSV)
    plt.xlabel('Time Period (seconds)')
    plt.ylabel('Pseudo Spectral Velocity')
    
    plt.figure()
    plt.plot(T, PSA)
    plt.xlabel('Time Period (seconds)')
    plt.ylabel('Pseudo Spectral Acceleration')
    
    plt.show()

# Example usage:
# Replace accel, ee, and dt with your actual values
# accel = np.random.randn(1000)
# ee = 5
# dt = 0.005
# responsespectrum(accel, ee, dt)
